# week_1: numpy 和 torch 基础

> 丁纪翔 20250227

In [4]:
import torch as th
import fasttext as ft
import sklearn as sk

In [5]:
th.cuda.is_available()

True

In [6]:
sk.__version__

'1.6.1'

## numpy基础用法

In [7]:
import numpy as np
np.__version__

'2.1.2'

In [8]:
# one-hot 编码
np.eye(4)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [9]:
# 随机数
np.random.random(5)

array([0.66652243, 0.24801062, 0.95404783, 0.52051314, 0.08707193])

In [10]:
np.random.normal(0, 1, (2, 3))


array([[ 1.08977527, -3.20318652,  0.6564581 ],
       [-0.47287886,  2.06576668,  0.28638856]])

In [11]:
a=np.arange(9).reshape(3,3)
a.ndim

2

In [12]:
a.shape

(3, 3)

In [13]:
a.size

9

In [14]:
a.dtype

dtype('int64')

In [15]:
3 in a

True

In [16]:
a

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [17]:
np.array([0,1,2]) in a

True

In [19]:
a.flatten()

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [21]:
b=np.random.randn(2,3)
b

array([[-0.81042832,  1.89053141,  0.52513146],
       [-1.21512643, -0.39525764, -0.97099303]])

In [23]:
b[None, ...].shape

(1, 2, 3)

In [24]:
import pandas as pd

In [25]:
pd.Series(b.flatten()).describe()

count    6.000000
mean    -0.162690
std      1.176728
min     -1.215126
25%     -0.930852
50%     -0.602843
75%      0.295034
max      1.890531
dtype: float64

In [27]:
b.prod()

np.float64(0.3752186391337539)

In [28]:
b.prod(axis=1)

array([-0.80457503, -0.46635631])

In [29]:
b.argmax()

np.int64(1)

In [30]:
np.rint(b) # 四舍五入

array([[-1.,  2.,  1.],
       [-1., -0., -1.]])

In [31]:
# 矩阵乘法
a=np.array([[1,2,],[3,4,]])
b=np.array([[5,6,],[7,8,]])
a@b

array([[19, 22],
       [43, 50]])

In [35]:
# 广播
dense_points = np.random.rand(10,25,3) # B, N, D
B, N, _ = dense_points.shape
# 从每个点云中随机选择5个点
selected_points = np.random.randint(0, N, (B, 5)) # B, 5
dense_points[np.arange(B)[:,None], selected_points].shape # B, 5, D



(10, 5, 3)

In [36]:
dense_points[[1,3],[3,4]].shape

(2, 3)

In [39]:
dense_points[np.array([1,3])[:,None],np.array([3,4])[None,:]].shape

(2, 2, 3)

广播发生的条件是：从尾部维度开始比较两个数组的形状，满足以下任一条件即可：

1. 维度相等。
2. 其中一个数组的维度为 1。
如果所有维度都满足以上条件，则可以进行广播。否则，将引发 ValueError 异常，提示形状不兼容。

举例说明：一个二维数组 (3, 4) 和一个一维数组 (4,) 可以进行广播。一维数组 (4,) 会被虚拟地扩展为 (1, 4)，然后再沿着第一个维度复制三次，变成 (3, 4)，从而与二维数组匹配。

## torch

In [40]:
import torch as th

In [41]:
th.tensor([1,2], dtype=th.float32)

tensor([1., 2.])

In [44]:
a=th.rand(3,4)
th.cat([a,a],dim=0).shape

torch.Size([6, 4])

In [45]:
th.stack([a,a]).shape

torch.Size([2, 3, 4])

In [46]:
a.device

device(type='cpu')

In [48]:
a.to('cuda:0')

tensor([[0.9593, 0.7312, 0.9850, 0.4830],
        [0.7937, 0.4126, 0.2218, 0.1517],
        [0.9877, 0.2798, 0.8407, 0.9329]], device='cuda:0')

In [49]:
!nvidia-smi

Thu Feb 27 09:57:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 561.09         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3060 Ti     On  |   00000000:01:00.0 Off |                  N/A |
|  0%   35C    P8             13W /  200W |     466MiB /   8192MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [50]:
a.add_(5) # 破坏计算图

tensor([[5.9593, 5.7312, 5.9850, 5.4830],
        [5.7937, 5.4126, 5.2218, 5.1517],
        [5.9877, 5.2798, 5.8407, 5.9329]])

In [57]:
from torchviz import make_dot

In [66]:
A = th.randn(2,2,requires_grad=True)
x = th.randn(1,2,requires_grad=True)
B = th.randn(2,2,requires_grad=True)

res = A @ x.T + B

dot = make_dot(res, params={'A': A, 'x': x, 'B': B})
dot.render('res', format='png', cleanup=True)

'res.png'

In [68]:
x.add_(x)

res_add_in_place = A @ x.T + B

dot_add_in_place = make_dot(res_add_in_place, params={'A': A, 'x': x, 'B': B})
dot_add_in_place.render('res_add_in_place', format='png', cleanup=True)


RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.

在 PyTorch 中，当一个张量设置了 requires_grad=True 时，对其进行 inplace（原地）操作会导致计算图的不一致性，这可能会影响自动微分的正确性。
1. 历史信息丢失：原始值被覆盖，反向传播时无法访问到原始值
2. 计算图断裂：依赖于原始张量的其他操作可能会引用到修改后的值

```python
# 不正确
x = torch.tensor([1.0], requires_grad=True)
x += 1  # inplace 操作

# 正确
x = torch.tensor([1.0], requires_grad=True)
x = x + 1  # 非 inplace 操作，创建新张量
```

计算 x + 1，创建一个新的张量对象作为结果，将这个新张量赋值给变量 x，原始的 x 张量对象仍然存在于内存中，直到没有引用指向它时才会被回收，计算图保持完整，因为创建了新的节点。